In [1]:
import torch
import torchvision
# from torch.utils.tensorboard import SummaryWriter
import time
from torch import nn
from torch.utils.data import DataLoader
from load_data import MyData  # self-made
from torchvision import transforms
from tqdm import tqdm_notebook as tqdm # View procedure
import os
import scipy.io
from random import random
import numpy as np
import gc
from torch.utils.tensorboard import SummaryWriter
from net_cnn_lstm1 import MyNetwork
from torchnlp.word_to_vector import GloVe
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [2]:
BATCH_SIZE = 1
C,H,W = 1,1,2400
# learn_rate = 0.0005
learn_rate=0.001
# num_epochs=50
num_epochs = 60

# ================================================ random
BATCH_SIZE = 1
C,H,W = 1,1,2400
# learn_rate = 0.0005
learn_rate=0.001
# num_epochs=50
num_epochs = 60

# ================================================ optimizer
import torch.optim as optim
device = torch.device("cuda:0")
gc.collect()
torch.cuda.empty_cache()
# ==损失函数权重
# ======== 二分类HC/DOC
# 计算总样本数量
# condition1
# total_samples = 887 + 985 + 879
# condition2
# total_samples = 929 + 1029 + 886
# condition3
# total_samples = 887 + 975 + 879
# rest
total_samples = 852 + 1051 + 872
# 计算每个类别的权重
# condition1
# weights = [total_samples / 887, total_samples / (985 + 879)]
# condition2
# weights = [total_samples / 929, total_samples / (1029 + 886)]
# condition3
weights = [total_samples / 852, total_samples / (1051 + 872)]

# ======== 二分类MCS/UWS
# 计算总样本数量
# condition1
# total_samples = 985 + 879
# condition2
# total_samples = 1029 + 886
# condition3
# total_samples = 975 + 879
# rest
# total_samples = 1051 + 872
# 计算每个类别的权重
# condition1
# weights = [total_samples / 985, total_samples / 879]
# condition2
# weights = [total_samples / 1029, total_samples / 886]
# condition3
# weights = [total_samples / 975, total_samples / 879]
# condition3
# weights = [total_samples / 1051, total_samples / 872]
# 将权重转换为张量
weights_tensor = torch.tensor(weights, device=device)

# 定义交叉熵损失函数并设置权重
criterion = nn.CrossEntropyLoss(weight=weights_tensor)
# criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)

In [3]:
def make_dataset(dataset,mode):
    # find the fold file
    count = 0
    for person in range(len(dataset)):
        filename = os.path.join(dataset.path, dataset.file_path[person])
        # extract the pure name of the file
        parts = filename.split("\\")
        file_name = parts[-1]
        name_without_extension = file_name.split(".")[0]
        # label or data
        file_last = name_without_extension.split("_")[-1]
        if file_last.isdigit(): # data
            # is this fold or not
            if int(file_last) == fold: # yes
                print(filename)
                count = count + 1
                data_map = torch.load(filename)
                # train or test
                if name_without_extension.split("_")[-2] == "train":
                    for i in range(data_map.size(0)):
                        train_data.append(data_map[i])
                elif name_without_extension.split("_")[-2] == "test":
                    for i in range(data_map.size(0)):
                        test_data.append(data_map[i])
                if count == 4:
                    del data_map
                    gc.collect()
                    torch.cuda.empty_cache() 
                    break
            else:   # not
                pass
        else: # label
            file_last = name_without_extension.split("_")[-4]
            file_mode = name_without_extension.split("_")[-2]
            if file_mode == mode.split("_")[-1]: # is this mode or not
                # is this fold or not
                if int(file_last) == fold: # yes
                    print(filename)
                    count = count + 1
                    data_map = torch.load(filename)
                    # train or test
                    if name_without_extension.split("_")[-5] == "train":
                        for i in range(data_map.size(0)):
                            train_label.append(data_map[i])
                    elif name_without_extension.split("_")[-5] == "test":
                        for i in range(data_map.size(0)):
                            test_label.append(data_map[i])
                    if count == 4:
                        del data_map
                        gc.collect()
                        torch.cuda.empty_cache() 
                        break
                else:   # not
                    pass
            else:
                pass
        del filename, parts, file_name, name_without_extension, file_last
        gc.collect()
        torch.cuda.empty_cache()    
# 定义LSTM超参数
input_size = 64  # 输入特征维度
# hidden_size = 64  # 隐藏单元数量
# input_size = 59  # 输入特征维度
hidden_size = 64  # 隐藏单元数量
num_layers = 2  # LSTM层数
output_size = 2  # 输出类别数量
# 创建模型实例
# model_list = ['', '_CNN', '_CNN_spa', '_CNN_spa_lstm']
# model_name = model_list[0]
# if model_name == model_list[0]: #  CascadeCept
#     from network_cnn_lstm import MyNetwork
# elif model_name == model_list[1]: #  CNN
#     from network_cnn_lstm_2 import MyNetwork
# elif model_name == model_list[2]: # CascadeCept_1
#     from network_cnn_lstm_3 import MyNetwork
# elif model_name == model_list[3]: # CascadeCept_2
#     from network_cnn_lstm_4 import MyNetwork
model_list = ['_1', '_2', '_3', '_4','_5','_6','_7','_8','_9','_10','_11']
model_name = model_list[0]
if model_name == model_list[0]: 
    from net_cnn_lstm1 import MyNetwork
elif model_name == model_list[1]: 
    from net_cnn_lstm2 import MyNetwork
elif model_name == model_list[2]: 
    from net_cnn_lstm3 import MyNetwork
elif model_name == model_list[3]: 
    from net_cnn_lstm4 import MyNetwork
elif model_name == model_list[4]: 
    from net_cnn_lstm5 import MyNetwork
elif model_name == model_list[5]: 
    from net_cnn_lstm6 import MyNetwork
elif model_name == model_list[6]: 
    from net_cnn_lstm7 import MyNetwork
elif model_name == model_list[7]: 
    from net_cnn_lstm8 import MyNetwork
elif model_name == model_list[8]: 
    from net_cnn_lstm9 import MyNetwork
elif model_name == model_list[9]: 
    from net_cnn_lstm10 import MyNetwork
elif model_name == model_list[10]: 
    from net_cnn_lstm11 import MyNetwork
model = MyNetwork(input_size, hidden_size, num_layers, output_size)
model = model.to(device)

# experimental dir: rest, conditionA, conditionB, conditionC
exper_dir = "condition1"
# ————————————————————————————————————————check this for every run!!!!!!!!!!!!!!!!
# trial="_1"
trial = ''
# root_dir = f"../data/eegmap_split/{exper_dir}"
root_dir = f"../data/eegmap_split/{exper_dir}"
# classification = "hc_doc"/doc or "mcs_uws"/uws
classification = "hc_doc"
# classification = "mcs_uws"
fold_num = 5
best_val_loss = float('inf')
for fold in tqdm(range(5)):
    # train num folds
#     fold = 0 # 选择折数
    # -- prepare datasets
    train_data = []
    train_label = []
    test_data = []
    test_label = []
    
    if classification == "hc_doc":
        # ---- hc
        dataset = MyData(root_dir, "train", "hc") # hc
        make_dataset(dataset,classification)
        dataset = MyData(root_dir, "test", "hc") # hc
        make_dataset(dataset,classification)
    # ---- mcs
    dataset = MyData(root_dir, "train", "mcs") # mcs
    make_dataset(dataset,classification)
    dataset = MyData(root_dir, "test", "mcs") # hc
    make_dataset(dataset,classification)
    # ---- uws
    dataset = MyData(root_dir, "train", "uws") # uws
    make_dataset(dataset,classification)
    dataset = MyData(root_dir, "test", "uws") # hc
    make_dataset(dataset,classification)
    
    print(torch.stack(train_data).size())
    print(torch.stack(train_label).size())
    print(torch.stack(test_data).size())
    print(torch.stack(test_label).size())
    del dataset
    gc.collect()
    torch.cuda.empty_cache()  
    
    train_data = torch.stack(train_data)
    train_label = torch.stack(train_label)
    test_data = torch.stack(test_data)
    test_label = torch.stack(test_label)
    # train dataset
    train_td = TensorDataset(train_data, train_label)
    train_loader = DataLoader(train_td, batch_size = BATCH_SIZE, shuffle = True)
    # test dataset
    test_td = TensorDataset(test_data, test_label)
    test_loader = DataLoader(test_td, batch_size = BATCH_SIZE, shuffle = True)
    del train_data, train_label, test_data, test_label, train_td, test_td
    gc.collect()
    torch.cuda.empty_cache()
    
    # set model for each fold
    model = MyNetwork(input_size, hidden_size, num_layers, output_size)
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr = learn_rate)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10, 30, 50], gamma=0.2)
#     scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)
    # -- start training
    start_time = time.time()
    # train and test step records
    total_train_step = 0
    total_test_step = 0
    min_test_loss = 1000
    # add Tensorboard
#     writer_train = SummaryWriter(f"../logs/{classification}/{exper_dir}{model_name}/logs_train_{fold}")
#     writer_valid = SummaryWriter(f"../logs/{classification}/{exper_dir}{model_name}/logs_test_{fold}")
#     writer_valid_acc = SummaryWriter(f"../logs/{classification}/{exper_dir}{model_name}/logs_test_acc_{fold}")
    writer_train = SummaryWriter(f"../logs/{classification}/{exper_dir}{model_name}{trial}/logs_train_{fold}")
    writer_valid = SummaryWriter(f"../logs/{classification}/{exper_dir}{model_name}{trial}/logs_test_{fold}")
    writer_valid_acc = SummaryWriter(f"../logs/{classification}/{exper_dir}{model_name}{trial}/logs_test_acc_{fold}")
    for i in tqdm(range(num_epochs)):  
        print(f"========= Epoch {i} Training =========")
        # train steps
        model.train()
        for data in train_loader:
            # x, y
            data_map, label=data
            data_map_reshaped = torch.reshape(data_map, (110, 1, 1, 2400))
#             data_map_reshaped = torch.reshape(data_map, (59, 1, 1, 2400))
            label_int = label.long()
            data_map_reshaped=data_map_reshaped.to(device)
            label_int=label_int.to(device)
            del data_map, label
            gc.collect()
            torch.cuda.empty_cache()
            # y_pred
            label_pred = model(data_map_reshaped)
            # Loss Computation and Optimization
            loss = criterion(label_pred,label_int)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # draw tensorboard
            total_train_step = total_train_step + 1
            # print info
            if total_train_step % 500 == 0:
                end_time = time.time()
                print(label_pred)
                print(f"Train time: {end_time - start_time}")
                print(f"Train steps: {total_train_step}, Loss: {loss.item()}")
            writer_train.add_scalar("train_loss",loss.item(),total_train_step)
            # Clear gpu
            del data, data_map_reshaped, label_int, label_pred, loss
            gc.collect()
            torch.cuda.empty_cache()
        
        # Evaluation and save the best model
        print(f"========= Epoch {i} Testing =========")
        model.eval()
        test_loss = 0
        total_test_loss = 0
        test_count = 0
        total_test_acc = 0
        with torch.no_grad():
            for data in test_loader:
                test_count = test_count + 1
                # x, y
                data_map, label=data
                data_map_reshaped = torch.reshape(data_map, (110, 1, 1, 2400))
#                 data_map_reshaped = torch.reshape(data_map, (59, 1, 1, 2400))
                label_int = label.long()
                data_map_reshaped = data_map_reshaped.to(device)
                label_int = label_int.to(device)
                del data_map, label
                gc.collect()
                torch.cuda.empty_cache()
                # y_pred
                label_pred_test = model(data_map_reshaped)
                loss = criterion(label_pred_test,label_int)
#                 print(label_pred_test)
                # accuracy 
                total_test_acc = total_test_acc + ((label_pred_test.argmax(1)) == label_int).sum()
                # change lr
                test_loss += loss.item()
                test_loss /= len(test_loader)
                if test_loss < best_val_loss:
                    best_val_loss = test_loss
                scheduler.step()
                # draw tensorboad
                total_test_loss = total_test_loss + loss
                if test_count % 100 == 0:
                    print(f"Loss: {total_test_loss} Accuracy: {total_test_acc/test_count}")
                # Clear gpu
                del data_map_reshaped, label_int, label_pred_test, loss, data
                gc.collect()
                torch.cuda.empty_cache()
        print(f"Total Loss: {total_test_loss} Total Accuracy: {total_test_acc/test_count}")
        writer_valid.add_scalar("test_loss", total_test_loss, total_test_step)
        writer_valid_acc.add_scalar("test_acc", total_test_acc/test_count, total_test_step)
        total_test_step = total_test_step + 1
        print("..........Saving the model..........")
#         torch.save(model.state_dict(),f"../models/{classification}/{exper_dir}{model_name}/Fold{fold}_Epoch{i}.pt") 
        torch.save(model.state_dict(),f"../models/{classification}/{exper_dir}{model_name}{trial}/Fold{fold}_Epoch{i}.pt") 
#         if total_test_loss < min_test_loss:
#             min_test_loss = total_test_loss
#             print("..........Saving the model..........")
#             torch.save(model.state_dict(),f"../model/{exper_dir}/Fold{fold}_Epoch{i}.pt")

D:\Anaconda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:117: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/5 [00:00<?, ?it/s]

../data/eegmap_split/condition1\train\hc\condition1_hc_train_0.pt
../data/eegmap_split/condition1\train\hc\condition1_hc_train_0_hc_doc_label.pt
../data/eegmap_split/condition1\test\hc\condition1_hc_test_0.pt
../data/eegmap_split/condition1\test\hc\condition1_hc_test_0_hc_doc_label.pt
../data/eegmap_split/condition1\train\mcs\condition1_mcs_train_0.pt
../data/eegmap_split/condition1\train\mcs\condition1_mcs_train_0_hc_doc_label.pt
../data/eegmap_split/condition1\test\mcs\condition1_mcs_test_0.pt
../data/eegmap_split/condition1\test\mcs\condition1_mcs_test_0_hc_doc_label.pt
../data/eegmap_split/condition1\train\uws\condition1_uws_train_0.pt
../data/eegmap_split/condition1\train\uws\condition1_uws_train_0_hc_doc_label.pt
../data/eegmap_split/condition1\test\uws\condition1_uws_test_0.pt
../data/eegmap_split/condition1\test\uws\condition1_uws_test_0_hc_doc_label.pt
torch.Size([3055, 2400, 10, 11])
torch.Size([3055])
torch.Size([762, 2400, 10, 11])
torch.Size([762])


D:\Anaconda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:184: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/60 [00:00<?, ?it/s]

...

tensor([[-5.0104,  5.5443]], device='cuda:0', grad_fn=<AddmmBackward0>)
Train time: 13080.1265335083
Train steps: 104500, Loss: 2.610649426060263e-05
tensor([[ 3.2068, -3.4893]], device='cuda:0', grad_fn=<AddmmBackward0>)
Train time: 13136.881619930267
Train steps: 105000, Loss: 0.0012349606258794665
tensor([[-4.8759,  5.4997]], device='cuda:0', grad_fn=<AddmmBackward0>)
Train time: 13193.337440252304
Train steps: 105500, Loss: 3.123234637314454e-05
tensor([[ 3.1440, -3.4596]], device='cuda:0', grad_fn=<AddmmBackward0>)
Train time: 13249.844747066498
Train steps: 106000, Loss: 0.0013546108966693282
tensor([[-4.8982,  5.5021]], device='cuda:0', grad_fn=<AddmmBackward0>)
Train time: 13306.34444117546
Train steps: 106500, Loss: 3.039790863113012e-05
========= Epoch 34 Testing =========
Loss: 47.15847396850586 Accuracy: 0.8999999761581421
Loss: 94.16275024414062 Accuracy: 0.8999999761581421
Loss: 135.0166015625 Accuracy: 0.9000000357627869
Loss: 190.75070190429688 Accuracy: 0.8924999833106

========= Epoch 39 Testing =========
Loss: 80.9375 Accuracy: 0.85999995470047
Loss: 133.056640625 Accuracy: 0.8799999952316284
Loss: 174.0369873046875 Accuracy: 0.8966667056083679
Loss: 207.3330078125 Accuracy: 0.9049999713897705
Loss: 251.9178924560547 Accuracy: 0.9020000696182251
Loss: 285.8402099609375 Accuracy: 0.9083333611488342
Loss: 341.1054992675781 Accuracy: 0.9071428179740906
Total Loss: 364.1429748535156 Total Accuracy: 0.9068241715431213
..........Saving the model..........
========= Epoch 40 Training =========
tensor([[ 4.2062, -4.4804]], device='cuda:0', grad_fn=<AddmmBackward0>)
Train time: 15435.895269870758
Train steps: 122500, Loss: 0.00016878610767889768
tensor([[ 4.5148, -4.8752]], device='cuda:0', grad_fn=<AddmmBackward0>)
Train time: 15492.613379478455
Train steps: 123000, Loss: 8.356221951544285e-05
tensor([[-5.3050,  5.9476]], device='cuda:0', grad_fn=<AddmmBackward0>)
Train time: 15549.383446931839
Train steps: 123500, Loss: 1.299372706853319e-05
tensor([[-4.69

tensor([[-6.2127,  6.8931]], device='cuda:0', grad_fn=<AddmmBackward0>)
Train time: 17562.241236925125
Train steps: 139000, Loss: 2.0265558760002023e-06
tensor([[ 5.3842, -5.6808]], device='cuda:0', grad_fn=<AddmmBackward0>)
Train time: 17618.596565008163
Train steps: 139500, Loss: 1.5616295058862306e-05
tensor([[ 5.1622, -5.4686]], device='cuda:0', grad_fn=<AddmmBackward0>)
Train time: 17675.042618751526
Train steps: 140000, Loss: 2.4199192921514623e-05
tensor([[ 5.0063, -5.2955]], device='cuda:0', grad_fn=<AddmmBackward0>)
Train time: 17731.530070781708
Train steps: 140500, Loss: 3.361645576660521e-05
========= Epoch 45 Testing =========
Loss: 85.78851318359375 Accuracy: 0.8899999856948853
Loss: 149.16981506347656 Accuracy: 0.8849999904632568
Loss: 231.66567993164062 Accuracy: 0.8700000047683716
Loss: 262.6794128417969 Accuracy: 0.8824999928474426
Loss: 342.914306640625 Accuracy: 0.878000020980835
Loss: 380.0069580078125 Accuracy: 0.8833333253860474
Loss: 418.8844909667969 Accuracy: 

Loss: 382.5086975097656 Accuracy: 0.9066666960716248
Loss: 462.8916931152344 Accuracy: 0.9057142734527588
Total Loss: 491.20928955078125 Total Accuracy: 0.9094488024711609
..........Saving the model..........
========= Epoch 51 Training =========
tensor([[ 6.1437, -6.5391]], device='cuda:0', grad_fn=<AddmmBackward0>)
Train time: 19803.051523447037
Train steps: 156000, Loss: 3.099436753473128e-06
tensor([[ 6.5735, -6.8363]], device='cuda:0', grad_fn=<AddmmBackward0>)
Train time: 19859.5703458786
Train steps: 156500, Loss: 1.5497195136049413e-06
tensor([[ 7.0114, -7.2705]], device='cuda:0', grad_fn=<AddmmBackward0>)
Train time: 19916.02338027954
Train steps: 157000, Loss: 5.960462772236497e-07
tensor([[-6.5673,  7.2275]], device='cuda:0', grad_fn=<AddmmBackward0>)
Train time: 19972.710739850998
Train steps: 157500, Loss: 1.0728830375228426e-06
tensor([[ 6.2929, -6.5572]], device='cuda:0', grad_fn=<AddmmBackward0>)
Train time: 20029.103978157043
Train steps: 158000, Loss: 2.62260073213838

tensor([[-8.0692,  8.7539]], device='cuda:0', grad_fn=<AddmmBackward0>)
Train time: 22048.231092453003
Train steps: 173500, Loss: 0.0
tensor([[ 8.4958, -8.7522]], device='cuda:0', grad_fn=<AddmmBackward0>)
Train time: 22104.907641649246
Train steps: 174000, Loss: 0.0
========= Epoch 56 Testing =========
Loss: 75.70745086669922 Accuracy: 0.9300000071525574
Loss: 147.08444213867188 Accuracy: 0.9199999570846558
Loss: 231.51153564453125 Accuracy: 0.9166666865348816
Loss: 375.5428771972656 Accuracy: 0.9049999713897705
Loss: 479.241943359375 Accuracy: 0.9080000519752502
Loss: 626.6896362304688 Accuracy: 0.9033333659172058
Loss: 713.5855102539062 Accuracy: 0.9085714221000671
Total Loss: 748.6110229492188 Total Accuracy: 0.9094488024711609
..........Saving the model..........
========= Epoch 57 Training =========
tensor([[ 8.5877, -8.8660]], device='cuda:0', grad_fn=<AddmmBackward0>)
Train time: 22215.015155792236
Train steps: 174500, Loss: 0.0
tensor([[-8.3170,  9.0529]], device='cuda:0', gra